# 🧹 DATA CLEANING AND MERGING PIPELINE
## Обработка исходных данных для прогнозирования кредитных дефолтов

---

### 📋 Что делает этот ноутбук:
1. ✅ Загружает **6 исходных файлов** разных форматов (CSV, Parquet, Excel, XML, JSONL)
2. ✅ Очищает данные (удаляет шум, исправляет форматирование)
3. ✅ Объединяет все датасеты в единый файл
4. ✅ Создает **базовые инженерные признаки** (финансовые коэффициенты, региональные индексы)
5. ✅ Разделяет на train/test (80/20, stratified)
6. ✅ Сохраняет очищенные датасеты

### 📂 Исходные файлы (из папки `drive-download-20251115T045945Z-1-001`):
1. `application_metadata.csv` (90,000 строк) - Заявки на кредит + целевая переменная `default`
2. `demographics.csv` (90,000 строк) - Демографические данные
3. `credit_history.parquet` (~90,000 строк) - Кредитная история
4. `financial_ratios.jsonl` (89,999 строк) - Финансовые показатели
5. `loan_details.xlsx` (Excel) - Детали кредита
6. `geographic_data.xml` (89,999 записей) - Географические данные

### 📦 Выходные файлы:
- `X_train_optimized.parquet` (71,999 × ~90 признаков)
- `X_test_optimized.parquet` (18,000 × ~90 признаков)
- `y_train.parquet`
- `y_test.parquet`

### ⏱️ Время выполнения: ~3-5 минут

---

## 1️⃣ Установка библиотек

In [ ]:
%%capture
!pip install -q pandas numpy scikit-learn pyarrow openpyxl

## 2️⃣ Импорты

In [ ]:
import pandas as pd
import numpy as np
import json
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

print("="*80)
print("DATA CLEANING AND MERGING PIPELINE")
print("Credit Default Prediction Dataset")
print("="*80)

## 3️⃣ Загрузка исходных файлов

**ВАЖНО:** Загрузите всю папку `drive-download-20251115T045945Z-1-001` с 6 файлами.

In [ ]:
from google.colab import files
import zipfile
import os

print("📤 Опция 1: Загрузите ZIP архив с данными")
print("   (Создайте ZIP из папки drive-download-20251115T045945Z-1-001)")
print()
print("📤 Опция 2: Загрузите 6 файлов вручную")
print()

uploaded = files.upload()

# If ZIP is uploaded, extract it
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        print(f"\n📦 Распаковка {filename}...")
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('.')
        print("✅ Распаковка завершена")

# Find data directory
if os.path.exists('drive-download-20251115T045945Z-1-001'):
    DATA_DIR = 'drive-download-20251115T045945Z-1-001'
else:
    DATA_DIR = '.'

print(f"\n📁 Директория с данными: {DATA_DIR}")
print(f"\n📋 Файлы в директории:")
for file in os.listdir(DATA_DIR):
    if not file.startswith('.'):
        size_mb = os.path.getsize(os.path.join(DATA_DIR, file)) / (1024 * 1024)
        print(f"   - {file} ({size_mb:.2f} MB)")

### 3.1 Загрузка application_metadata.csv

In [ ]:
print("\n[1/6] Loading application_metadata.csv...")

app_metadata = pd.read_csv(f"{DATA_DIR}/application_metadata.csv")

print(f"✓ Shape: {app_metadata.shape}")
print(f"✓ Target variable 'default' distribution:")
print(f"   Non-default (0): {(app_metadata['default'] == 0).sum():,} ({(app_metadata['default'] == 0).mean()*100:.2f}%)")
print(f"   Default (1):     {(app_metadata['default'] == 1).sum():,} ({(app_metadata['default'] == 1).mean()*100:.2f}%)")
print(f"\n✓ Sample:")
print(app_metadata.head(3))

### 3.2 Загрузка demographics.csv

In [ ]:
print("\n[2/6] Loading demographics.csv...")

demographics = pd.read_csv(f"{DATA_DIR}/demographics.csv")

print(f"✓ Shape: {demographics.shape}")
print(f"\n⚠️ Data Quality Issues Detected:")
print(f"   annual_income formatting: {demographics['annual_income'].head(5).tolist()}")
print(f"   employment_type inconsistency: {demographics['employment_type'].value_counts().head().to_dict()}")
print(f"\n✓ Sample:")
print(demographics.head(3))

### 3.3 Загрузка credit_history.parquet

In [ ]:
print("\n[3/6] Loading credit_history.parquet...")

credit_history = pd.read_parquet(f"{DATA_DIR}/credit_history.parquet")

print(f"✓ Shape: {credit_history.shape}")
print(f"✓ Missing values: {credit_history.isnull().sum().sum()}")
print(f"\n✓ Sample:")
print(credit_history.head(3))

### 3.4 Загрузка financial_ratios.jsonl

In [ ]:
print("\n[4/6] Loading financial_ratios.jsonl...")

financial_ratios = []
with open(f"{DATA_DIR}/financial_ratios.jsonl", 'r') as f:
    for line in f:
        financial_ratios.append(json.loads(line))
financial_ratios = pd.DataFrame(financial_ratios)

print(f"✓ Shape: {financial_ratios.shape}")
print(f"\n⚠️ Data Quality Issues Detected:")
print(f"   monthly_income formatting: {financial_ratios['monthly_income'].head(5).tolist()}")
print(f"   (Contains $, commas - need cleaning)")
print(f"\n✓ Sample:")
print(financial_ratios.head(3))

### 3.5 Загрузка loan_details.xlsx

In [ ]:
print("\n[5/6] Loading loan_details.xlsx...")

loan_details = pd.read_excel(f"{DATA_DIR}/loan_details.xlsx")

print(f"✓ Shape: {loan_details.shape}")
print(f"\n✓ Sample:")
print(loan_details.head(3))

### 3.6 Загрузка geographic_data.xml

In [ ]:
print("\n[6/6] Loading geographic_data.xml...")

tree = ET.parse(f"{DATA_DIR}/geographic_data.xml")
root = tree.getroot()

# Parse XML records
geo_records = []
for customer in root.findall('customer'):
    record = {}
    for child in customer:
        record[child.tag] = child.text
    geo_records.append(record)

geographic_data = pd.DataFrame(geo_records)

print(f"✓ Shape: {geographic_data.shape}")
print(f"\n✓ Sample:")
print(geographic_data.head(3))

print("\n" + "="*80)
print("✅ ALL 6 FILES LOADED SUCCESSFULLY")
print("="*80)

## 4️⃣ ОЧИСТКА ДАННЫХ

Исправление проблем качества данных в каждом файле.

### 4.1 Очистка application_metadata

In [ ]:
print("\n[Cleaning 1/6] application_metadata...")

# Remove noise column
print("  Removing 'random_noise_1' column...")
app_metadata_clean = app_metadata.drop(columns=['random_noise_1'])

print(f"  ✓ Shape after cleaning: {app_metadata_clean.shape}")
print(f"  ✓ Noise column removed")

### 4.2 Очистка demographics

In [ ]:
print("\n[Cleaning 2/6] demographics...")

demographics_clean = demographics.copy()

# Clean annual_income (remove $, commas)
print("  Cleaning annual_income (removing $, commas)...")
demographics_clean['annual_income'] = (
    demographics_clean['annual_income']
    .str.replace('$', '', regex=False)
    .str.replace(',', '', regex=False)
    .astype(float)
)
print(f"    Before: {demographics['annual_income'].head(3).tolist()}")
print(f"    After:  {demographics_clean['annual_income'].head(3).tolist()}")

# Normalize employment_type
print("\n  Normalizing employment_type...")
def normalize_employment_type(emp_type):
    emp_upper = str(emp_type).upper()
    if 'FULL' in emp_upper or 'FT' in emp_upper:
        return 'Full-time'
    elif 'PART' in emp_upper or 'PT' in emp_upper:
        return 'Part-time'
    elif 'SELF' in emp_upper or 'CONTRACT' in emp_upper:
        return 'Self-employed'
    else:
        return 'Other'

demographics_clean['employment_type'] = demographics_clean['employment_type'].apply(normalize_employment_type)
print(f"    Normalized categories: {demographics_clean['employment_type'].value_counts().to_dict()}")

# Impute missing employment_length
missing_emp_len = demographics_clean['employment_length'].isnull().sum()
if missing_emp_len > 0:
    print(f"\n  Imputing {missing_emp_len} missing employment_length with 0...")
    demographics_clean['employment_length'] = demographics_clean['employment_length'].fillna(0)

print(f"\n  ✓ Demographics cleaned: {demographics_clean.shape}")

### 4.3 Очистка credit_history

In [ ]:
print("\n[Cleaning 3/6] credit_history...")

credit_history_clean = credit_history.copy()

# Impute missing num_delinquencies_2yrs
missing_delinq = credit_history_clean['num_delinquencies_2yrs'].isnull().sum()
if missing_delinq > 0:
    print(f"  Imputing {missing_delinq} missing num_delinquencies_2yrs with 0...")
    credit_history_clean['num_delinquencies_2yrs'] = credit_history_clean['num_delinquencies_2yrs'].fillna(0)

print(f"  ✓ Credit history cleaned: {credit_history_clean.shape}")

### 4.4 Очистка financial_ratios

In [ ]:
print("\n[Cleaning 4/6] financial_ratios...")

financial_ratios_clean = financial_ratios.copy()

# Function to clean monetary fields
def clean_monetary_field(series):
    return (
        series.astype(str)
        .str.replace('$', '', regex=False)
        .str.replace(',', '', regex=False)
        .replace('nan', np.nan)
        .astype(float)
    )

# Clean all monetary fields
monetary_columns = ['monthly_income', 'existing_monthly_debt', 'monthly_payment',
                   'revolving_balance', 'credit_usage_amount', 'available_credit',
                   'total_monthly_debt_payment', 'total_debt_amount', 'monthly_free_cash_flow']

print("  Cleaning monetary columns...")
for col in monetary_columns:
    if col in financial_ratios_clean.columns:
        print(f"    {col}: ", end="")
        sample_before = str(financial_ratios_clean[col].iloc[0])
        financial_ratios_clean[col] = clean_monetary_field(financial_ratios_clean[col])
        sample_after = financial_ratios_clean[col].iloc[0]
        print(f"{sample_before} → {sample_after}")

# Impute missing revolving_balance
missing_revolv = financial_ratios_clean['revolving_balance'].isnull().sum()
if missing_revolv > 0:
    print(f"\n  Imputing {missing_revolv} missing revolving_balance with 0...")
    financial_ratios_clean['revolving_balance'] = financial_ratios_clean['revolving_balance'].fillna(0)

print(f"\n  ✓ Financial ratios cleaned: {financial_ratios_clean.shape}")

### 4.5 Очистка loan_details

In [ ]:
print("\n[Cleaning 5/6] loan_details...")

loan_details_clean = loan_details.copy()

# Clean loan_amount if it has monetary formatting
if loan_details_clean['loan_amount'].dtype == 'object':
    print("  Cleaning loan_amount...")
    loan_details_clean['loan_amount'] = clean_monetary_field(loan_details_clean['loan_amount'])

# Normalize loan_type
print("  Normalizing loan_type...")
def normalize_loan_type(loan_type):
    loan_upper = str(loan_type).upper()
    if 'PERSONAL' in loan_upper:
        return 'Personal'
    elif 'MORTGAGE' in loan_upper or 'HOME' in loan_upper:
        return 'Mortgage'
    elif 'CREDIT' in loan_upper or 'CC' in loan_upper:
        return 'Credit Card'
    elif 'AUTO' in loan_upper or 'CAR' in loan_upper:
        return 'Auto'
    else:
        return 'Other'

loan_details_clean['loan_type'] = loan_details_clean['loan_type'].apply(normalize_loan_type)
print(f"    Categories: {loan_details_clean['loan_type'].value_counts().to_dict()}")

print(f"\n  ✓ Loan details cleaned: {loan_details_clean.shape}")

### 4.6 Очистка geographic_data

In [ ]:
print("\n[Cleaning 6/6] geographic_data...")

geographic_data_clean = geographic_data.copy()

# Convert numeric columns
numeric_geo_cols = ['regional_unemployment_rate', 'regional_median_income',
                   'regional_median_rent', 'housing_price_index', 'cost_of_living_index']

print("  Converting to numeric...")
for col in numeric_geo_cols:
    if col in geographic_data_clean.columns:
        geographic_data_clean[col] = pd.to_numeric(geographic_data_clean[col], errors='coerce')
        print(f"    {col}: ✓")

print(f"\n  ✓ Geographic data cleaned: {geographic_data_clean.shape}")

print("\n" + "="*80)
print("✅ ALL DATA CLEANED SUCCESSFULLY")
print("="*80)

## 5️⃣ ОБЪЕДИНЕНИЕ ДАТАСЕТОВ

Объединение всех 6 файлов в один датасет.

In [ ]:
print("\n" + "="*80)
print("MERGING ALL DATASETS")
print("="*80 + "\n")

# Start with application_metadata (contains target)
merged = app_metadata_clean.copy()
print(f"[1] Starting with application_metadata: {merged.shape}")

# Merge demographics
print(f"[2] Merging demographics...")
merged = merged.merge(
    demographics_clean,
    left_on='customer_ref',
    right_on='cust_id',
    how='left',
    validate='1:1'
)
print(f"    After merge: {merged.shape}")

# Merge credit_history
print(f"[3] Merging credit_history...")
merged = merged.merge(
    credit_history_clean,
    left_on='customer_ref',
    right_on='customer_number',
    how='left',
    validate='1:1'
)
print(f"    After merge: {merged.shape}")

# Merge financial_ratios
print(f"[4] Merging financial_ratios...")
merged = merged.merge(
    financial_ratios_clean,
    left_on='customer_ref',
    right_on='cust_num',
    how='left',
    validate='1:1'
)
print(f"    After merge: {merged.shape}")

# Merge loan_details
print(f"[5] Merging loan_details...")
merged = merged.merge(
    loan_details_clean,
    left_on='customer_ref',
    right_on='customer_id',
    how='left',
    validate='1:1'
)
print(f"    After merge: {merged.shape}")

# Merge geographic_data
print(f"[6] Merging geographic_data...")
geographic_data_clean['id'] = geographic_data_clean['id'].astype(int)
merged = merged.merge(
    geographic_data_clean,
    left_on='customer_ref',
    right_on='id',
    how='left',
    validate='1:1'
)
print(f"    After merge: {merged.shape}")

# Drop redundant ID columns
print(f"\n[7] Dropping redundant ID columns...")
id_columns = ['cust_id', 'customer_number', 'cust_num', 'customer_id', 'id']
merged = merged.drop(columns=[col for col in id_columns if col in merged.columns])
print(f"    Final shape: {merged.shape}")

print("\n✅ ALL DATASETS MERGED SUCCESSFULLY")
print(f"   Total records: {merged.shape[0]:,}")
print(f"   Total features: {merged.shape[1]:,}")

## 6️⃣ ИНЖЕНЕРИЯ ПРИЗНАКОВ

Создание дополнительных признаков для улучшения модели.

In [ ]:
print("\n" + "="*80)
print("FEATURE ENGINEERING")
print("="*80 + "\n")

# Income-based features
print("[1] Creating income-based features...")
merged['monthly_income_from_annual'] = merged['annual_income'] / 12
merged['disposable_income'] = merged['monthly_income'] - merged['existing_monthly_debt']
merged['income_to_payment_capacity'] = merged['monthly_income'] / (merged['monthly_payment'] + 1)
print("    ✓ Created 3 features")

# Debt burden features
print("[2] Creating debt burden features...")
merged['total_debt_to_income_annual'] = merged['total_debt_amount'] / (merged['annual_income'] + 1)
merged['debt_payment_burden'] = (
    (merged['existing_monthly_debt'] + merged['monthly_payment']) / (merged['monthly_income'] + 1)
)
merged['free_cash_flow_ratio'] = merged['monthly_free_cash_flow'] / (merged['monthly_income'] + 1)
merged['loan_to_monthly_income'] = merged['loan_amount'] / (merged['monthly_income'] + 1)
print("    ✓ Created 4 features")

# Credit behavior features
print("[3] Creating credit behavior features...")
merged['credit_age_to_score_ratio'] = merged['oldest_account_age_months'] / (merged['credit_score'] + 1)
merged['delinquency_rate'] = merged['num_delinquencies_2yrs'] / (merged['num_credit_accounts'] + 1)
merged['inquiry_intensity'] = merged['num_inquiries_6mo'] + merged['recent_inquiry_count']
merged['negative_marks_total'] = (
    merged['num_delinquencies_2yrs'].fillna(0) +
    merged['num_public_records'] +
    merged['num_collections']
)
merged['credit_stress_score'] = (
    merged['credit_utilization'] * 0.3 +
    merged['debt_to_income_ratio'] * 0.3 +
    merged['delinquency_rate'] * 0.4
)
print("    ✓ Created 5 features")

# Loan characteristics
print("[4] Creating loan characteristic features...")
merged['loan_amount_to_limit'] = merged['loan_amount'] / (merged['total_credit_limit'] + 1)
merged['interest_burden'] = merged['loan_amount'] * merged['interest_rate'] / 100
merged['loan_term_years'] = merged['loan_term'] / 12
merged['monthly_loan_payment_estimate'] = merged['loan_amount'] / (merged['loan_term'] + 1)
print("    ✓ Created 4 features")

# Regional economic features
print("[5] Creating regional economic features...")
merged['income_to_regional_median'] = merged['annual_income'] / (merged['regional_median_income'] + 1)
merged['housing_affordability'] = merged['regional_median_rent'] / (merged['monthly_income'] + 1)
merged['regional_stress_index'] = (
    merged['regional_unemployment_rate'] * 0.4 +
    (merged['cost_of_living_index'] / 100) * 0.3 +
    (merged['housing_price_index'] / 100) * 0.3
)
print("    ✓ Created 3 features")

# Behavioral features
print("[6] Creating behavioral features...")
merged['service_call_intensity'] = merged['num_customer_service_calls'] / (merged['num_login_sessions'] + 1)
merged['digital_engagement_score'] = (
    merged['has_mobile_app'] * 0.5 +
    merged['paperless_billing'] * 0.3 +
    (merged['num_login_sessions'] / merged['num_login_sessions'].max()) * 0.2
)
print("    ✓ Created 2 features")

# Application timing features
print("[7] Creating application timing features...")
merged['is_business_hours'] = ((merged['application_hour'] >= 9) & (merged['application_hour'] <= 17)).astype(int)
merged['is_weekend'] = (merged['application_day_of_week'].isin([6, 7])).astype(int)
merged['is_late_night'] = ((merged['application_hour'] >= 22) | (merged['application_hour'] <= 5)).astype(int)
print("    ✓ Created 3 features")

# Account maturity
print("[8] Creating account maturity features...")
current_year = 2025
merged['account_age_years'] = current_year - merged['account_open_year']
merged['credit_history_depth'] = (
    merged['oldest_credit_line_age'] * 0.5 +
    merged['oldest_account_age_months'] * 0.5
)
print("    ✓ Created 2 features")

print(f"\n✅ FEATURE ENGINEERING COMPLETE")
print(f"   Total new features: 26")
print(f"   Dataset shape: {merged.shape}")

## 7️⃣ ОБРАБОТКА ПРОПУЩЕННЫХ ЗНАЧЕНИЙ

In [ ]:
print("\n" + "="*80)
print("HANDLING MISSING VALUES")
print("="*80 + "\n")

missing_summary = merged.isnull().sum()
missing_summary = missing_summary[missing_summary > 0].sort_values(ascending=False)

if len(missing_summary) > 0:
    print("Columns with missing values:")
    for col, count in missing_summary.items():
        pct = (count / len(merged)) * 100
        print(f"  {col}: {count:,} ({pct:.2f}%)")
    print("\nImputing with 0 (conservative approach for financial data)...")
    merged = merged.fillna(0)
    print("✓ All missing values imputed")
else:
    print("✓ No missing values detected")

print(f"\nFinal shape: {merged.shape}")

## 8️⃣ РАЗДЕЛЕНИЕ НА TRAIN/TEST

In [ ]:
print("\n" + "="*80)
print("TRAIN/TEST SPLIT")
print("="*80 + "\n")

# Separate features and target
X = merged.drop(columns=['default'])
y = merged['default']

# Stratified split to preserve class balance
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=RANDOM_SEED
)

print(f"Training set: {X_train.shape[0]:,} samples ({(X_train.shape[0]/len(X))*100:.1f}%)")
print(f"  Default rate: {y_train.mean()*100:.2f}%")
print(f"\nTest set: {X_test.shape[0]:,} samples ({(X_test.shape[0]/len(X))*100:.1f}%)")
print(f"  Default rate: {y_test.mean()*100:.2f}%")
print(f"\n✓ Class balance preserved in both sets")

## 9️⃣ КОДИРОВАНИЕ КАТЕГОРИАЛЬНЫХ ПЕРЕМЕННЫХ

In [ ]:
print("\n" + "="*80)
print("ENCODING CATEGORICAL VARIABLES")
print("="*80 + "\n")

# Identify categorical columns
categorical_cols = X_train.select_dtypes(include=['object']).columns.tolist()
print(f"Categorical columns: {len(categorical_cols)}")
for col in categorical_cols:
    print(f"  - {col}")

# Label encode (fit on combined train+test to handle all categories)
print("\nLabel encoding...")
label_encoders = {}
for col in categorical_cols:
    all_categories = pd.concat([X_train[col], X_test[col]]).astype(str).unique()
    le = LabelEncoder()
    le.fit(all_categories)
    X_train[col] = le.transform(X_train[col].astype(str))
    X_test[col] = le.transform(X_test[col].astype(str))
    label_encoders[col] = le
    print(f"  {col}: {len(le.classes_)} classes → [0-{len(le.classes_)-1}]")

print("\n✓ All categorical variables encoded")

## 🔟 СОХРАНЕНИЕ ДАТАСЕТОВ

In [ ]:
print("\n" + "="*80)
print("SAVING DATASETS")
print("="*80 + "\n")

# Save cleaned datasets
X_train.to_parquet('X_train_optimized.parquet', index=False)
X_test.to_parquet('X_test_optimized.parquet', index=False)
pd.DataFrame({'default': y_train}).to_parquet('y_train.parquet', index=False)
pd.DataFrame({'default': y_test}).to_parquet('y_test.parquet', index=False)

print("✓ Saved files:")
print(f"  - X_train_optimized.parquet: {X_train.shape}")
print(f"  - X_test_optimized.parquet: {X_test.shape}")
print(f"  - y_train.parquet: {len(y_train):,} records")
print(f"  - y_test.parquet: {len(y_test):,} records")

print("\n" + "="*80)
print("✅ DATA CLEANING AND MERGING COMPLETE")
print("="*80)
print("\nNext steps:")
print("  1. Download the 4 parquet files")
print("  2. Upload to Data_Preparation_Colab_V3.ipynb")
print("  3. Create leak-free OOF features")
print("  4. Train XGBoost model")
print("="*80)

## 📊 ИТОГОВАЯ СТАТИСТИКА

In [ ]:
print("\n" + "="*80)
print("FINAL STATISTICS")
print("="*80)

print(f"\n📊 Data Summary:")
print(f"  Total records processed: {merged.shape[0]:,}")
print(f"  Total features created: {merged.shape[1]:,}")
print(f"  Training samples: {X_train.shape[0]:,}")
print(f"  Test samples: {X_test.shape[0]:,}")
print(f"  Class imbalance ratio: {(y_train == 0).sum() / (y_train == 1).sum():.1f}:1")

print(f"\n🔧 Feature Engineering:")
print(f"  Income-based: 3 features")
print(f"  Debt burden: 4 features")
print(f"  Credit behavior: 5 features")
print(f"  Loan characteristics: 4 features")
print(f"  Regional economic: 3 features")
print(f"  Behavioral: 2 features")
print(f"  Application timing: 3 features")
print(f"  Account maturity: 2 features")
print(f"  TOTAL ENGINEERED: 26 features")

print(f"\n✅ Data Quality:")
print(f"  ✓ No missing values")
print(f"  ✓ No noise columns")
print(f"  ✓ All monetary fields cleaned")
print(f"  ✓ Categorical variables normalized")
print(f"  ✓ Class balance preserved in split")

print("\n" + "="*80)

## 💾 Download Files

In [ ]:
from google.colab import files

print("📥 Downloading files...\n")

files.download('X_train_optimized.parquet')
files.download('X_test_optimized.parquet')
files.download('y_train.parquet')
files.download('y_test.parquet')

print("\n✅ All files downloaded!")
print("\n🎯 Use these files in Data_Preparation_Colab_V3.ipynb for OOF feature engineering")

---

## 📋 Summary

### Completed Tasks:
1. ✅ Loaded 6 heterogeneous data files (CSV, Parquet, Excel, XML, JSONL)
2. ✅ Cleaned all data quality issues:
   - Removed `random_noise_1` column
   - Cleaned monetary formatting ($, commas)
   - Normalized categorical variables
   - Imputed missing values
3. ✅ Merged all datasets into single unified file
4. ✅ Created 26 engineered features
5. ✅ Performed stratified 80/20 train/test split
6. ✅ Encoded categorical variables
7. ✅ Saved cleaned datasets

### Output Files:
- `X_train_optimized.parquet` - Clean training features
- `X_test_optimized.parquet` - Clean test features
- `y_train.parquet` - Training labels
- `y_test.parquet` - Test labels

### Next Steps:
1. Use `Data_Preparation_Colab_V3.ipynb` to create leak-free OOF features
2. Use `Google_Colab_Leak_Free_90plus_v3.ipynb` to train XGBoost model
3. Achieve 90%+ AUC!

---

**Author:** Data Pipeline Engineer  
**Version:** V1 - Data Cleaning & Merging  
**Date:** November 2025

---